# Logistic Regression

## Hi!
In today's workshop we are going to learn about most known concept of supervised learning which is **classification**.

### What is classification?
Classification is a problem of predicting discrete value (classes) for given features. It is mainly viewed as a supervised learning problem.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mlxtend.data import loadlocal_mnist

from ipywidgets import interact, fixed
import ipywidgets as widgets

import solutions

%load_ext autoreload
%autoreload 2

Just like last time, we'll work with a very real-world dataset describing a couple hundred cases of breast cancer, which presents an example of a case for **binary classification**

In [ ]:
print(load_breast_cancer().DESCR)

First, we'll split our data int train, test, and validation datasets

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
np.random.seed(0)
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=0.7)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, train_size=0.66)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

### What about applying linear regression for classification?

Let's take a look at the target data:

In [ ]:
y

It's a bunch of ones and zeros! Wouldn't it make sense to just train a linear regressor on the data?

In [ ]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

In [ ]:
X_train[0]

In [ ]:
X_val.shape

In [ ]:
linear_reg.predict(X_val)

How to interpret these predictions? Maybe we need something different?

![classification_regression](img/clas_reg.png)

### What is logistic regression?

Logistic regression is about applying a "squashing" function to the hypotheses when calculating loss.

### $$h_w(x) = \sum_{j=0}^k w_j x_j = wx$$

### $$\hat{y} = \sigma(h_w(x))$$ 

## Why do we need squashing?

### One of such squashing functions is sigmoid function:
### $$\sigma(x) = \frac{1}{1+e^{-x}}$$

In [ ]:
x = np.linspace(-10, 10)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
plt.plot(x, sigmoid(x))
plt.grid(True)
plt.show()

In [ ]:
sigmoid(np.inf), sigmoid(-np.inf)

### Because of non-linearities in our hypotheses, we also need to update our loss function.

We'll use a logarythmic loss function which quite nicely captures an intuition, that we want the predictions datapoins which should be predicted as $0$ as close to $0$ as possible, and, analogically, predictions which should be $1$, as close to $1$ as possible:

### $$ L(w) = \frac{-1}{n}(\sum_{i=0}^n y^{(i)}\log{h_w(x^{(i)})} + (1-y^{(i)})\log{(1-h_w(x^{(i)}))} )$$

### $$ y^{(i)} \in \{0, 1\}$$

In [ ]:
# y = 0
x = np.linspace(0, 0.9999, 1000)
plt.plot(x, -np.log(1 - x))
plt.ylim(-1, 10)
plt.show()

In [ ]:
# y = 1
x = np.linspace(0.0001, 1, 1000)
plt.plot(x, -np.log(x))
plt.ylim(-1, 10)
plt.show()

Let's try and implement this new loss function!

In [ ]:
def loss(
    W: np.ndarray, 
    X: np.ndarray, 
    Y: np.ndarray, 
    eps: float = 0.01 # the epsilon parameter is for numeric stability of logarithm
) -> float:
    return 0

In [ ]:
loss = solutions.loss

In [ ]:
W = np.random.rand(X.shape[1])
print(loss(W, X, y, eps=0.1))
print(solutions.loss(W, X, y, eps=0.1))

What about gradient descent procedure? How does it change? Let's derive the gradient!

[we'll do that on the board] 
It turns out, it's very simple!

### $$
\frac{\partial L(W)}{\partial W} =\frac{1}{n}(\sum_{i=0}^n x^{(i)} \cdot (h_w(x^{(i)}) - y^{(i)}))
$$

In [ ]:
def gradient_step(
    W, 
    X, 
    Y,
    learning_rate=0.01
) -> np.ndarray:
    return np.zeros_like(W)

In [ ]:
gradient_step = solutions.gradient_step

In [ ]:
W = np.random.rand(X.shape[1])

yours = gradient_step(W, X, y, learning_rate=0.1)
provided = solutions.gradient_step(W, X, y, learning_rate=0.1)
print(yours - provided)

Let's not forget about adding the bias feature and normalizing the data!

In [ ]:
def add_bias_feature(X):
       return np.c_[np.ones(len(X)), X]

In [ ]:
X_train = add_bias_feature(X_train)
X_val = add_bias_feature(X_val)
X_test = add_bias_feature(X_test)

In [ ]:
X_train, *norm_parameters = solutions.std_normalization(X_train)
X_val, *_ = solutions.std_normalization(X_val, *norm_parameters)
X_test, *_ = solutions.std_normalization(X_test, *norm_parameters)

In [ ]:
np.random.seed(0)
W = np.random.randn(X_train.shape[1])
train_costs = []
val_costs = []
train_steps = 100
for _ in range(train_steps):
    train_costs.append(loss(W, X_train, y_train, eps=0.001))
    val_costs.append(loss(W, X_val, y_val, eps=0.001))
    W = gradient_step(W, X_train, y_train, learning_rate=0.1)
   

In [ ]:
plt.plot(np.arange(train_steps), train_costs)
plt.plot(np.arange(train_steps), val_costs)
plt.show()

In [ ]:
accuracy_score(y_train, solutions._hypotheses(W, X_train) >= 0.5)

In [ ]:
accuracy_score(y_val, solutions._hypotheses(W, X_val) >= 0.5)

How does our model compare to the one provided by Scikit-Learn?

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_reg = LogisticRegression(C=10**6)

In [ ]:
logistic_reg.fit(X_train, y_train)

In [ ]:
logistic_reg.score(X_train, y_train)

In [ ]:
logistic_reg.score(X_val, y_val)

### A great score! Or is it?

In [ ]:
positive_ind = np.argwhere(y_val == 1).reshape(-1)
negative_ind = np.argwhere(y_val == 0).reshape(-1)
X_val_pos = X_val[positive_ind]
y_val_pos = y_val[positive_ind]
X_val_neg = X_val[negative_ind]
y_val_neg = y_val[negative_ind]

In [ ]:
accuracy_score(y_val_pos, solutions._hypotheses(W, X_val_pos) >= 0.5)

In [ ]:
accuracy_score(y_val_neg, solutions._hypotheses(W, X_val_neg) >= 0.5)

We achieve higher accuracies on positive examples, than on negative ones. In practice, this means we're likelier to classify tumors as malignant than not. 

Better safe than sorry? Turns out, not always. Can we dig deeper into the performance of our model?

### Precision and recall
We can divide classifications of our model into four classes:

| Predicted/Actual | 0   | 1   |
|------------------|-----|-----|
| 0                | True negative | False negative|
| 1                | False positive | True positive | 


**Accuracy - a first intuition**

$$
Accuracy = \frac{T_p + T_n}{T_n + T_p + F_n + F_p}
$$

However, as we've just seen, this metric may be deceiving (consider class imbalance!)

Turns out there is a more reliable way to measure the performance of our model:

- **Precision** - *what fraction of our positive classifications is correct?*
$$
Precision = \frac{T_p}{T_p + F_p}
$$

- **Recall** - *what fraction of actual positive examples has been classified correctly?*
$$
Recall = \frac{T_p}{T_p + F_n}
$$

We want both of those values to be as high as possible (duh).

However, sometimes we have to make a trade off between them and decide with our classification method that one will be higher and the other lower.

A metric which nicely mixes the two above is called the **F1 score** - it's high when both precision and recall are high enough, but low when one of them is sacrificed for the sake of another.

$$
F1 = \frac{2PR}{P +R}
$$

#### Can precision and recall be manipulated without tinkering with the model?

In [ ]:
def calc_precision_recall(
    X: np.ndarray,
    y: np.ndarray,
    W: np.ndarray,
    threshold: float
):
    print("threshold", threshold)
    y_pred = solutions._hypotheses(W, X)
    y_pred_bin = y_pred >= threshold
    print('precision', precision_score(y, y_pred_bin))
    print('recall', recall_score(y, y_pred_bin))
    print('F1 score', f1_score(y, y_pred_bin))
    positive_ind = np.argwhere(y == 1).reshape(-1)
    negative_ind = np.argwhere(y == 0).reshape(-1)
    y_pos = y[positive_ind]
    y_neg = y[negative_ind]
    y_pos_pred = y_pred_bin[positive_ind]
    y_neg_pred = y_pred_bin[negative_ind]
    print('total accuracy', accuracy_score(y, y_pred_bin))
    print('positive accuracy', accuracy_score(y_pos, y_pos_pred))
    print('negative accuracy', accuracy_score(y_neg, y_neg_pred))

In [ ]:
interact(
    calc_precision_recall,
    X=fixed(X_val),
    y=fixed(y_val),
    W=fixed(W),
    threshold=widgets.FloatSlider(
        value=0.5,
        min=0,
        max=1,
        step=0.01
    )
)

#### How does F1 score depend on the threshhold?

In [ ]:
thresholds = np.linspace(.01, .99, 100)
scores = []

for t in thresholds:
    y_pred = solutions._hypotheses(W, X_val)
    y_pred_bin = y_pred >= t
    scores.append(f1_score(y_val, y_pred_bin))
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.grid(True)
plt.plot(thresholds, scores)
plt.xlabel("Threshold")
plt.ylabel("F1 score")
plt.show()

#### To better visualize how precision and recall depend on each other, we can also plot an AUROC curve

**A**rea

**U**nder

**R**eceiver

**O**perating

**C**haracteristic

In [ ]:
thresholds = np.linspace(.01, .99, 100)
precisions = []
recalls = []

for t in thresholds:
    y_pred = solutions._hypotheses(W, X_val)
    y_pred_bin = y_pred >= t
    precisions.append(precision_score(y_val, y_pred_bin))
    recalls.append(recall_score(y_val, y_pred_bin))
plt.xlim(0, 1.2)
plt.ylim(0, 1.2)
plt.grid(True)
plt.plot(precisions, recalls)
plt.xlabel("Precision")
plt.ylabel("Recall")
plt.show()

## Sometimes, binary classification is not enough!

https://www.youtube.com/watch?v=pqTntG1RXSY

### Multiclass classification

To solve the problem of classifying an object as one of multiple classes, we do a one-vs-all prediction. 
Previously we calculated $h_w(x)$ and applied $sigmoid$ function to it, to calculate the 'probablility' of our example being positive or not. Since $\hat{y} \in [0,1]$, we chose a 'threshold' in that range below which we can treat our example as negative and above which - as positive.

For multiple classes, we must essentially calculate a hypothesis for **every single one** of possible categories. If hypothesis for a given category is high enough, there is a high probability that our object is of that category. In the other case, it means that it belongs to some other category (but we don't know which one - we need other hypotheses for that). 

![alt text](img/multiclass.PNG)
This is called one-versus-all classification. Ultimately we choose the category whose hypothesis has the highest probability.



### Before

Until now, a hypothesis $h_w(x^{(i)})$ for a given object $x^{(i)}$ represented as a vector of features $[x_0^{(i)}, x_1^{(i)}, ... x_k^{(i)}]$ was represented by a scalar:

$$h_w(x) = \sigma(\sum_{j=0}^k w_j x_j) = \sigma(wx)$$

Where w was a vector a weights. 

### Now

If $m$ is the number of possible categories, then for every vector of features we want to perform multiple logistic regressions (for every possible category we might classify it as). 
Essentially, for every vector of $k$ features we now want to obtain a vector of $m$ hypothesis scalars:

$$[x_0^{(i)}, x_1^{(i)}, ... x_k^{(i)}] \xrightarrow{\text{classification}} [h_0^{(i)}, h_1^{(i)}, ... h_m^{(i)}]$$


For every logistic regression we need a separate $k$-dimensional vector (or a $k \times 1$ matrix) of weights. 
If we want to vectorize our computations, we can merge all of the weights vectors into a single, $k \times m$ matrix.

$X$ - an $n \times k$ matrix representing the examples
$$
X = \begin{bmatrix}
x_0^{(1)} & x_1^{(1)}  &  & ...  &x_k^{(1)}\\ 
x_0^{(2)} &...  &  &...  & \\ 
... &  &  &...  & \\ 
x_0^{(n)} &  & ... &  & x_k^{(n)}
\end{bmatrix}
$$

$W$ - an $k \times m$ matrix representing weights in logistic regression for every feature in every category

\begin{bmatrix}
w_0^{(1)} & w_0^{(2)}  &  & ...  &w_0^{(m)}\\ 
w_1^{(1)} &...  &  &...  & \\ 
... &  &  &...  & \\ 
w_k^{(1)} &  & ... &  & w_k^{(m)}
\end{bmatrix}


$h_W(X)$ - an $n \times m$ matrix representing hypothesis vectors for every example and category 

$$
h_W(X) = \sigma(XW)
$$

We'll denote j-th hypothesis of i-th example as $$h_w^{(j)}(x^{(i)})$$
Computationally-wise, the only thing that changes is the $m$ dimension of W.


### As for cost function...

$$ L^{(j)}(w) = -\sum_{i=0}^n y^{(i,j)}\log{h_w^{(j)}(x^{(i)})} + (1-y^{(i,j)})\log{(1-h_w(x^{(i)}))}$$

We have a vector of cost values for every category $j$, which is useful in updating weights in gradient descent. If we want to plot the cost function, we can sum or count the mean of all those values.

Gradient descent also works the same way as before.

#### WTF is $y^{(i,j)}$?

We can now look at y as a matrix of one-hot values. If $y^{(i,j)} = 1 $, then example $i$ is of class $j$. 

This also means the rest of values in $y^{(i)}$ are, of course, zeros.



In [ ]:
hypotheses = solutions._hypotheses
loss = solutions.loss
gradient_step = solutions.gradient_step

### MNIST - something more ambitious

MNIST is one of the most famous datasets for beginers in Machine Learning.

In [ ]:
if not os.path.exists("train-images-idx3-ubyte"):
    !curl -O http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
    !curl -O http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
    !curl -O http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
    !curl -O http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
    !gunzip t*-ubyte.gz

In [ ]:
X1, y1 = loadlocal_mnist(
        images_path="train-images-idx3-ubyte", 
        labels_path="train-labels-idx1-ubyte")

X2, y2 = loadlocal_mnist(
        images_path="t10k-images-idx3-ubyte", 
        labels_path="t10k-labels-idx1-ubyte")

X = np.concatenate([X1, X2])
y = np.concatenate([y1, y2])

In [ ]:
X[0]

Image of a digit can be visualized as array of $784 (= 28*28)$ numbers, or just a picture. 
For convenience values of pixels are stored not as a 2D array, but as a vector, so in order to be displayed, the vector must be reshaped.

In [ ]:
pixels = img.reshape(28,28) / 255
plt.imshow(pixels, cmap='gray')
plt.show()
y[0]

We are going to treat every single pixel as a separate feature. In order to do so, let's normalize them. We'll also create one-hot vectors of labels we can fit our model to.

In [ ]:
examples_count = X.shape[0]
labels = y
normalized_pixels_nobias = X / 255
one_hot_labels = np.zeros((examples_count, 10))
one_hot_labels[np.arange(examples_count), labels] = 1
one_hot_labels[0]

In [ ]:
def display_mnist_elem(index):
    img = X[rand_no]
    pixels = img.reshape(28,28) / 255
    plt.imshow(pixels, cmap='gray')
    plt.show()
    print('label:', labels[rand_no])
    print('label as a one-hot vector:', one_hot_labels[rand_no])

In [ ]:
examples_count = normalized_pixels_nobias.shape[0]
rand_no = np.random.randint(0, examples_count)
display_mnist_elem(rand_no)

In [ ]:
normalized_pixels = add_bias_feature(normalized_pixels_nobias) 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_pixels, one_hot_labels)

Training the classifier will work the same as in binary classification!

In [ ]:
W = np.random.random((785,10)) # 784 + bias feature
losses = []
steps = 100

for i in range(steps):
    W = gradient_step(W, X_train, Y_train, learning_rate=0.1)
    losses.append(loss(W, X_train, Y_train))

plt.plot(np.arange(steps), losses)
plt.show()

Observing loss decrease is one thing, but let's see our classifier in action!

In [ ]:
rand_no = np.random.randint(examples_count) # we choose a random digit from dataset
display_mnist_elem(rand_no)
img_pixels = normalized_pixels[rand_no]
predicted_H = hypotheses(W, img_pixels)
predicted_class = np.argmax(predicted_H)

print('predicted hypotheses:', predicted_H)
print('predicted_class:', predicted_class)

In [ ]:
H_test = hypotheses(W, X_test)
predicted_test_labels = np.argmax(H_test, axis=1)
accuracy_score(np.argmax(Y_test, axis=1), predicted_test_labels)

For MNIST it's actually embarassingly bad (best models achieve even 99.9% accuracy), but it's not so bad for one matrix!

## Next time, Neural Networks! :O

Spoiler alert!

![spoiler](img/lr_spoiler.jpg)